In [2]:
print("ok")

ok


In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os

In [29]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [5]:
# EXTRACT HTE DATA FROM PDF
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents=loader.load()
    return documents

In [6]:
extracted_data = load_pdf("data/")

In [7]:
#extracted_data

In [8]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [10]:
# text_chunks

In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [14]:
query_result=embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
#query_result

In [18]:

from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'medical-bot'
index = pc.Index(index_name)

#creating embeddings for each of the text chunks and storing
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [19]:
query="Define Electrostatic Potential"
docs=docsearch.similarity_search(query,k=3)
print(docs)

[Document(page_content='a patient is in advanced stages of the disease.\nElectrophoresis —A method of separating complex\nprotein molecules suspended in a gel by running anelectric current through the gel.\nEnzyme-linked immunosorbent assay (ELISA) —A'), Document(page_content='impulses from the heart. The resulting graph of electricalactivity can show if the heart muscle isn’t functioningproperly as a result of a lack of oxygen. Electrocardio-grams are also useful in investigating other possibleabnormal features of the heart.\nStress test\nFor many individuals with angina, the results of an'), Document(page_content='contribute to an episode of angina.\nDiagnosis\nPhysicians can usually diagnose angina based on the\npatient’s symptoms and the precipitating factors. Howev-er, other diagnostic testing is often required to confirm orrule out angina, or to determine the severity of the under-lying heart disease.\nElectrocardiogram (ECG)\nAn electrocardiogram is a test that records electrica

In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer ,just say that you don't know ,don't try to makeup an answer.

Context:{context}
Question:{question}

Only return the helpful answer below and nothing else

Helpful answer:
"""

In [21]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

In [22]:
llm=CTransformers(model="/Users/piyushpandey955/Desktop/AIML/gen ai/medical chatbot project/Medical-Chatbot/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8}
                )

In [23]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
import pinecone

# Use the Pinecone vectorstore from Langchain
vectorstore = Pinecone(
    index=pinecone.Index(index_name, host='https://medical-chatbot-9d82pte.svc.aped-4627-b74a.pinecone.io'),
    embedding_function=embeddings.embed_query,
    text_key="text"
    )

# Set up the retriever with Pinecone and Langchain
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [24]:
user_input = "what is Acne?"
query_embedding = embeddings.embed_query(user_input)
len(query_embedding)


384

In [27]:

# Query Pinecone using the keyword arguments format
result = index.query(vector=query_embedding, top_k=2, include_metadata=True)

# Check if results are found
if result and 'matches' in result:
    # Loop through the top results
    for match in result['matches']:
        # Extract metadata if available (assuming text is stored in metadata)
        if 'metadata' in match and 'text' in match['metadata']:
            # Extract and print the relevant text
            print("Match:", match['metadata']['text'])
        else:
            print("No metadata or text field found in this match.")
else:
    print("No results found.")

Match: Acidosis seeRespiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is
Match: GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne
Acne vulgaris affecting a woman’s face. Acne is the general
name given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-
ates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25
